<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/filtra_deter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pacotes
import pathlib
from osgeo import ogr, gdal, gdalconst
from skimage.morphology import disk, dilation, erosion
import os
import numpy as np
import shutil
from google.colab import drive
import cv2
from datetime import date, timedelta
import pandas as pd

In [2]:
# Conectando ao drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/drive/My Drive/projeto_DL/'
os.chdir(root_path)

Mounted at /content/drive


In [3]:
def converte_deter_tif(path_to_deter, output_path, patch_name, x_size, y_size, extent_tuple, d_begin, d_end):
  # Abrindo arquivo
  v_deter = ogr.Open(path_to_deter)
  l_deter = v_deter.GetLayer()
  # Definindo localização
  x_res = (extent_tuple[2]-extent_tuple[0])/x_size
  y_res = (extent_tuple[3]-extent_tuple[1])/y_size
  geo_transform = [extent_tuple[0], x_res, 0, extent_tuple[1], 0, y_res]
  # Nome de saída
  output = os.path.join(output_path, f'{patch_name}.tif')
  # Filtro de datas sendo aplicado
  where_ref = f'(cast(\"VIEW_DATE\" as character) <\'{d_end}\') AND ( cast(\"VIEW_DATE\" as character) >= \'{d_begin}\' )'
  # Objeto que manipula os dados
  target_ds = gdal.GetDriverByName('GTiff').Create(output, x_size, y_size, 1, gdal.GDT_Byte)
  target_ds.SetGeoTransform(geo_transform)
  l_deter.ResetReading()
  l_deter.SetAttributeFilter(where_ref)
  print (l_deter.GetFeatureCount())
  # Salvando
  gdal.RasterizeLayer(target_ds, [1], l_deter, burn_values=[1])
  # Limpando
  target_ds.FlushCache()
  del target_ds
  #
  return 1

In [4]:
# Parâmetros fixos
output_path='/content/drive/My Drive/projeto_DL/deter_tratado/'
deterFile="/content/drive/My Drive/projeto_DL/dados_brutos/deter_public.shp"
x_size = 256*10
y_size = 256*10
extent= [-59.9206710380000018, -7.3125828444699836, -59.1668605229999969, -6.7566813625300162]

In [5]:
# # Rodando em loop - versao mensal
#
# # Construindo intervalos de datas
# anos_rodar = ['2015','2016','2017','2018','2019','2020','2021', '2022']
# l_d_begin = []
# for i in anos_rodar:
#   for j in ['01','02','03','04','05','06','07','08','09','10','11','12']:
#     l_d_begin.append(i+"/"+j+"/"+"01")
#
# l_d_end = l_d_begin.copy()
# del l_d_end[0]
# l_d_end.append('2022/01/01')

# # Loop
# for i in range(len(l_d_begin)):
#   patch_name=f"{i}"
#   d_begin = l_d_begin[i]
#   d_end = l_d_end[i]
#   converte_deter_tif(deterFile, output_path + 'mensal/', patch_name, x_size, y_size, extent, d_begin, d_end)
#   print((i+1)/len(l_d_begin))

In [6]:
# pd.DataFrame({'begin':l_d_begin, 'end':l_d_end, 'patch_name':np.arange(len(l_d_end))}).to_csv('/content/drive/My Drive/projeto_DL/deter_tratado/referencia_mes.csv')

In [7]:
# Rodando em loop - versao semanal

# Pegando todos os domingos do ano
# https://stackoverflow.com/questions/2003870/how-can-i-select-all-of-the-sundays-for-a-year-using-python
def all_wdays(year, wday = 6):
   d = date(year, 1, 1)                    # January 1st
   d += timedelta(days = wday - d.weekday())  # 0 : Monday ... 6 : Sunday
   if d.year != year:
    d += timedelta(days = 7)
   while d.year == year:
      yield d
      d += timedelta(days = 7)

# Montando datas
# Começo da semana
anos_rodar = [2015,2016,2017,2018,2019,2020,2021,2022]
l_d_begin = []
for i in anos_rodar:
  for d in all_wdays(i):
    l_d_begin.append(d)

# Fim da semana
l_d_end = []
for i in l_d_begin:
  l_d_end.append(i+timedelta(days = 6))

# Loop
for i in range(len(l_d_begin)):
  patch_name=f"{i}"
  d_begin = l_d_begin[i].strftime("%Y/%m/%d")
  d_end = l_d_end[i].strftime("%Y/%m/%d")
  converte_deter_tif(deterFile, output_path + 'semanal/', patch_name, x_size, y_size, extent, d_begin, d_end)
  if ((i+1) % 100) == 0:
    print((i+1)/len(l_d_begin))


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1804
685
2486
703
904
2313
210
850
2371
2241
1176
2673
363
1692
349
196
228
848
0.23980815347721823
44
4
0
10
14
64
106
62
0
80
0
120
0
149
20
24
37
5
146
143
22
141
545
27
366
71
849
242
452
717
679
170
462
385
247
268
204
554
604
1029
2126
2416
535
2055
758
864
773
912
509
419
254
452
678
395
693
334
136
177
639
353
211
124
762
41
581
244
226
1031
249
369
55
653
1070
459
0
109
1129
741
223
586
360
1466
943
895
954
1312
853
813
1386
925
609
1693
1422
1605
733
464
276
886
756
827
0.47961630695443647
210
102
295
0
0
251
114
0
96
65
123
51
200
238
68
245
31
289
347
315
235
287
227
121
132
76
995
185
572
470
627
629
856
512
921
2228
1979
2348
2325
1600
2387
3301
2773
2089
2416
3453
1282
1090
691
329
557
890
861
652
92
964
445
156
193
119
107
323
341
415
79
87
129
212
43
93
405
0
199
469
419
60
163
769
247
264
444
868
370
1347
1

In [9]:
pd.DataFrame({'begin':l_d_begin, 'end':l_d_end, 'name':np.arange(len(l_d_end))}).to_csv('/content/drive/My Drive/projeto_DL/deter_tratado/referencia_semana.csv')

In [10]:
# Checando
path_checar = "/content/drive/My Drive/projeto_DL/deter_tratado/semanal/"
print(len(os.listdir(path_checar))==len(l_d_begin))

True
